In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

batch_size = 64
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

valid_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)

print("Dataset downloaded")

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(28*28, 512) 
        self.fc2 = nn.Linear(512, 64) 
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        
        x = x.view(-1, 28*28) 
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
model = MLP()
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Neural Network defined")

In [ ]:
for epoch in range(10): 

    #training
    model.train()
    for images, labels in train_loader:
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    #evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Epoch = {epoch+1}, Validation set accuracy: {accuracy:.2f}%')